# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

## Q1
The test that is most appropriate for this situation is the two-sample t-test. A two-sample t-test is used to test the difference between two population means. A common application is to determine whether the means are equal between those two groups, which in this case is 'black-sounding' and 'white-sounding'. 

More specifically, we're going to test whether these two groups have similar call back rates from prospective employers.

Additionally, the Central Limit Theorem tells us that if we take the mean of the samples (n) and plot the frequencies of their mean, we will get a normal distribution. Also, as the sample size (n) increases, the distribution will look more and more like a normal distribution. 

The sample size (n) has to be large (usually n >= 30) if the population from where the same is taken is non-normal. If the population follows the normal distribution then the sample size n can be either small or large. Based off of this, as long as we keep our sample size (n) greater than 30 than the Central Limit Thereom will hold. 

## Q2
We're trying to determine if there is a difference between the two population means (i.e. d = 0). When the null hypothesis states that there is no difference the null and alternative hypothesis are often stated as follows: 

- Ho: μ1 = μ2

- Ha: μ1 ≠ μ2

With Ho being the null hypothesis and Ha being the alternative hypothesis. 

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

# Solution to Q3

In [24]:
# black-sounding and white-sounding population statistics (mean and standard deviation)
mean_w = (np.sum(w['call']) / len(w['call']))
mean_b = (np.sum(b['call']) / len(b['call']))
std_w = np.std(w['call'])
std_b = np.std(b['call'])
print('The mean callback proportion of the white-sounding population is equal to ', mean_w)
print('The mean callback proportion of the black-sounding population is equal to ', mean_b)
print('The callback proportions standard deviation from the white-sounding population is equal to ', std_w)
print('The callback proportions standard deviation from the black-sounding population is equal to ', std_b)

The mean callback proportion of the white-sounding population is equal to  0.09650924024640657
The mean callback proportion of the black-sounding population is equal to  0.06447638603696099
The callback proportions standard deviation from the white-sounding population is equal to  0.29528486728668213
The callback proportions standard deviation from the black-sounding population is equal to  0.24559901654720306


In [15]:
# function to draw random sample, of size that is equal to the length of the data
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

# function to store means of the bootstrap replicates, with default size of 1 
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [29]:
# sample of size n = 100 of callback proportion from white-sounding population
sample_w = draw_bs_reps(w['call'], np.mean, 100)

In [30]:
print('The mean callback proportion of the white-sounding sample is equal to ', np.mean(sample_w))
print('The callback proportions standard deviation from the white-sounding sample is equal to ', np.std(sample_w))

The mean callback proportion of the white-sounding sample is equal to  0.09665708392858505
The callback proportions standard deviation from the white-sounding sample is equal to  0.005623290667187303


In [32]:
# sample means of size n = 100 of callback proportion from black-sounding population
sample_b = draw_bs_reps(b['call'], np.mean, 100)

In [33]:
print('The mean callback proportion of the black-sounding sample is equal to ', np.mean(sample_b))
print('The callback proportions standard deviation from the black-sounding sample is equal to ', np.std(sample_b))

The mean callback proportion of the black-sounding sample is equal to  0.0646940452232957
The callback proportions standard deviation from the black-sounding sample is equal to  0.005376413184229845


In [41]:
# standard deviation of the samples
sample_w_std = np.sqrt((np.mean(sample_w) * (1 - np.mean(sample_w)) / 100))
print(sample_w_std)

sample_b_std = np.sqrt((np.mean(sample_b) * (1 - np.mean(sample_b)) / 100))
print(sample_b_std)

0.029549025712366134
0.02459852144661176


In [39]:
# mean of the difference of the samples
sample_difference_mean = np.mean(sample_w) + np.mean(sample_b) / 2
sample_difference_mean

0.1290041065402329

In [42]:
# standard deviation of the difference of the samples
sample_difference_std = sample_w_std + sample_b_std
sample_difference_std

0.05414754715897789

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>